# Sample Maps with shared utils

In [ ]:
import branca
import geopandas as gpd
import pandas as pd

import shared_utils 

In [ ]:
shared_utils.geography_utils.CA_NAD83Albers

In [ ]:
GEOSPATIAL_DATA = "https://opendata.arcgis.com/datasets/a0f1da19bffe471ea89bd6a241e5604b_0.geojson"
gdf = gpd.read_file(GEOSPATIAL_DATA)
gdf

In [ ]:
print(f"original CRS: {gdf.crs}")
gdf = gdf.to_crs(shared_utils.geography_utils.CA_StatePlane)
print(f"new CRS: {gdf.crs}")

In [ ]:
shared_utils.utils.geoparquet_gcs_export?

In [ ]:
GCS_FILE_PATH = "gs://calitp-analytics-data/"
FILE_NAME = "test_geoparquet"
shared_utils.utils.geoparquet_gcs_export(gdf, GCS_FILE_PATH, FILE_NAME)

In [ ]:
popup_dict = {
    "DISTRICT": "Caltrans District",
    "Region": "Caltrans Region",
}

'''
# Set your own step colormap and specify colors and cut-offs
colorscale = branca.colormap.StepColormap(
                colors=["gray", "green", "navy"], 
                index=[2_000, 4_000, 8_000],
                vmin=0, vmax=15_000,
)

# Set your own linear colormap and specify colors and cut-offs
colorscale = branca.colormap.LinearColormap(
                colors=["gray", "green", "navy"], 
                index=[2_000, 4_000, 8_000],
                vmin=0, vmax=15_000,
)

# Use preset step colormap
colorscale = branca.colormap.step.Accent_03

# Use preset linear colormap
colorscale = branca.colormap.linear.Blues_06

'''
colorscale = (branca.colormap.step.Accent_07
              .scale(vmin=gdf.DISTRICT.min(), 
                     vmax=gdf.DISTRICT.max())
             )

m = shared_utils.map_utils.make_folium_choropleth_map(
    gdf,
    plot_col = "DISTRICT",
    popup_dict = popup_dict,
    tooltip_dict = popup_dict,
    colorscale = colorscale,
    fig_width = 500, fig_height = 800,
    zoom = shared_utils.map_utils.REGION_CENTROIDS["CA"][1],
    centroid = shared_utils.map_utils.REGION_CENTROIDS["CA"][0],
    title="CA Map"
)

#m

In [ ]:
import json

gdf = gdf.to_crs(shared_utils.geography_utils.WGS84)

geo_data = json.loads(gdf.set_index("DISTRICT").to_json())
choro_data = dict(zip(gdf["DISTRICT"].tolist(), gdf["Shape_Area"].tolist()))

choropleth_dict = {
    "geo_data": geo_data,
    "choro_data": choro_data,
    "layer_name": "Area",
    "MIN_VALUE": gdf.Shape_Area.min(),
    "MAX_VALUE": gdf.Shape_Area.max(),
    "plot_col_name": "Area (sq mi)",
    "fig_width": '100%',
    "fig_height": '100%',
    "fig_min_width_px": '400px',
    "fig_min_height_px": '600px',
}

colorscale = branca.colormap.linear.Greens_09

f = shared_utils.map_utils.make_ipyleaflet_choropleth_map(
    choropleth_dict, 
    plot_col = "Shape_Area",
    colorscale = colorscale
)

f